# Automação Web e Busca de Informações com Python

### Desafio: 

- Importação dos preço dos nossos produtos vinculados à cotação de **dólar, ouro e euro**;

- Extração de dados da internet, de forma automática, para a cotação desses 3 itens e saber quanto deve-se cobrar pelos produtos, considerando uma margem de contribuição. 

### Informações disponíveis 
- Planilha de produtos contendo nome dos insumos gerais que serão atualizados com informações da internet.

### Etapas do projeto

- Extração das cotações de dólar, euro e ouro;
- Atualizar a planilha de dados com as cotações atualizados;
- Compilação dos dados em tabela excel (.xlsx). 

### Instalação da biblioteca _Selenium_

- A biblioteca selenium é uma das principais ferramentas para recolhimentos de informações na internet (**_Web Scrapping_**). Caso não tenha instalado no seu computador pessoal, é necessário executar o comando "pip install --upgrade selenium" dentro do prompt do anaconda;
- Importante verificar a versão do navegador (aconselhável Google Chrome), e acessar a página https://chromedriver.chromium.org/downloads para baixar a versão compatível com o navegador; 
- Após fazer o download do arquivo contendo a versão do _chromedriver_ compatível com seu computador pessoal, é necessário descompactar o arquivo .zip e colar na pasta pessoal do anaconda3. 

In [10]:
# Importação das bibliotecas para criação do navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# Biblioteca para manipulação dos dados
import pandas as pd 
import numpy as np

# Modificando o diretório do trabalho
import os 
path = r'C:\Users\cador\OneDrive\Área de Trabalho\Projetos Github\Python - Web Scrapping\WS - Cotacao moeda'
os.chdir(path)

# abrir um navegador
navegador = webdriver.Chrome()

### Passo 1: Extrair cotação do dólar dentro do navegador

In [11]:
# abrir um navegador
navegador.get("https://www.google.com/")

# Pesquisa do nome do produto no google- Cópia do caminho XPATH 
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar", Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") 

display(cotacao_dolar)

'5.1479'

### Passo 2: Extrair cotação do euro dentro do navegador

In [12]:
# Pesquisa do nome do produto no google- Cópia do caminho XPATH 

navegador.get("https://www.google.com/")

navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro", Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

display(cotacao_euro)

'5.226405475'

### Passo 3: Extrair cotação do ouro dentro do navegador

In [13]:
# Pesquisa do nome do produto no google- Cópia do caminho XPATH 
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()

285.88


### Atualização da base de preços com as novas cotações extraídas da internet

- Importação a base de dados

In [14]:
tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualização dos preços e o cálculo do Preço Final

In [15]:
# Passo 5: Recalcular o preço de cada produto
# atualizar a cotação
# nas linhas onde na coluna "Moeda" = Dólar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o preço base reais (preço base original * cotação)
tabela["Preço de Compra"] = np.round (tabela["Preço Original"] * tabela["Cotação"], 2)

# atualizar o preço final (preço base reais * Margem)
tabela["Preço de Venda"] = np.round(tabela["Preço de Compra"] * tabela["Margem"], 2)

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.147900,5147.85,1.40,7206.99
1,Carro Renault,4500.00,Euro,5.226405,23518.82,2.00,47037.64
2,Notebook Dell,899.99,Dólar,5.147900,4633.06,1.70,7876.20
3,IPhone,799.00,Dólar,5.147900,4113.17,1.70,6992.39
4,Carro Fiat,3000.00,Euro,5.226405,15679.22,1.90,29790.52
5,Celular Xiaomi,480.48,Dólar,5.147900,2473.46,2.00,4946.92
6,Joia 20g,20.00,Ouro,285.880000,5717.60,1.15,6575.24


###  Exportar a nova base de preços atualizada

In [16]:
# Passo 6: Salvar os novos preços dos produtos
tabela.to_excel("Produtos Novo.xlsx", index=False)